<a href="https://colab.research.google.com/github/njain05/urban_parking_dynamic_pricing/blob/main/final_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [176]:
!pip install pathway bokeh --quiet

In [177]:
# Dynamic Pricing for Urban Parking Lots
# Capstone Project Implementation
# ============================================================================
# IMPORTS AND SETUP
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Bokeh imports
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot, column, row
from bokeh.models import HoverTool, ColumnDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import Category10, Spectral6, Viridis256
from bokeh.transform import factor_cmap
from bokeh.io import push_notebook
import bokeh.palettes as bp

# Enable Bokeh notebook output
output_notebook()

print("✅ Libraries imported successfully!")
print("📋 Next: Run the Data Processing cell")


✅ Libraries imported successfully!
📋 Next: Run the Data Processing cell


In [178]:
# ============================================================================
# DATA PROCESSING CLASS
# ============================================================================

class ParkingDataProcessor:
    def __init__(self, data_path=None):
        self.data = None
        self.parking_lots = None
        self.base_price = 10.0

    def load_data(self, data_path):
        """Load and preprocess the parking data"""
        try:
            self.data = pd.read_csv(data_path)
            print(f"✅ Data loaded successfully: {self.data.shape}")
            print(f"📊 Columns: {list(self.data.columns)}")
            return True
        except Exception as e:
            print(f"❌ Error loading data: {e}")
            return False

    def preprocess_data(self):
        """Clean and prepare data for modeling"""
        if self.data is None:
            print("❌ No data loaded. Please load data first.")
            return False

        print("🔄 Starting data preprocessing...")

        # Create datetime column with correct format
        self.data['DateTime'] = pd.to_datetime(
            self.data['LastUpdatedDate'] + ' ' + self.data['LastUpdatedTime'],
            format='%d-%m-%Y %H:%M:%S'
        )
        print("✅ DateTime column created")

        # Calculate occupancy rate
        self.data['OccupancyRate'] = self.data['Occupancy'] / self.data['Capacity']
        print("✅ Occupancy rate calculated")

        # Encode categorical variables
        self.data['VehicleType_encoded'] = self.data['VehicleType'].map({
            'car': 1.0, 'bike': 0.5, 'truck': 1.5, 'cycle': 0.3
        })

        self.data['TrafficCondition_encoded'] = self.data['TrafficConditionNearby'].map({
            'low': 0.5, 'average': 1.0, 'high': 1.5
        })
        print("✅ Categorical variables encoded")

        # Extract time features
        self.data['Hour'] = self.data['DateTime'].dt.hour
        self.data['DayOfWeek'] = self.data['DateTime'].dt.dayofweek
        print("✅ Time features extracted")

        # Get unique parking lots
        self.parking_lots = self.data['SystemCodeNumber'].unique()
        print(f"✅ Found {len(self.parking_lots)} unique parking lots")

        print("🎉 Preprocessing complete!")
        return True

    def explore_data(self):
        """Basic data exploration"""
        if self.data is None:
            print("❌ No data to explore")
            return

        print("\n" + "="*50)
        print("📊 DATA EXPLORATION SUMMARY")
        print("="*50)

        print(f"📈 Dataset shape: {self.data.shape}")
        print(f"📅 Date range: {self.data['DateTime'].min()} to {self.data['DateTime'].max()}")
        print(f"🏢 Number of parking lots: {len(self.parking_lots)}")

        print(f"\n🚗 Parking Lots: {list(self.parking_lots)}")

        print("\n📊 Occupancy Statistics:")
        print(self.data['OccupancyRate'].describe().round(3))

        print("\n🚙 Vehicle Type Distribution:")
        print(self.data['VehicleType'].value_counts())

        print("\n🚦 Traffic Condition Distribution:")
        print(self.data['TrafficConditionNearby'].value_counts())

        print("\n🎯 Special Days:")
        print(f"Regular days: {(self.data['IsSpecialDay'] == 0).sum()}")
        print(f"Special days: {(self.data['IsSpecialDay'] == 1).sum()}")

print("✅ Data Processing Class defined!")


✅ Data Processing Class defined!


In [180]:
# ============================================================================
# MODEL 1 - BASELINE LINEAR MODEL
# ============================================================================

class BaselineLinearModel:
    def __init__(self, base_price=10.0, alpha=5.0):
        self.base_price = base_price
        self.alpha = alpha  # Sensitivity parameter
        self.current_prices = {}
        self.model_name = "Baseline Linear Model"

    def initialize_prices(self, parking_lots):
        """Initialize prices for all parking lots"""
        for lot in parking_lots:
            self.current_prices[lot] = self.base_price
        print(f"✅ {self.model_name}: Initialized prices for {len(parking_lots)} lots")

    def predict_price(self, lot_id, occupancy_rate, previous_price=None):
        """
        Simple linear pricing model
        Price(t+1) = Price(t) + α * (Occupancy/Capacity)
        """
        if previous_price is None:
            previous_price = self.current_prices.get(lot_id, self.base_price)

        # Linear price adjustment based on occupancy
        price_adjustment = self.alpha * occupancy_rate
        new_price = previous_price + price_adjustment

        # Apply bounds (0.5x to 2x base price)
        new_price = max(self.base_price * 0.5, min(new_price, self.base_price * 2.0))

        self.current_prices[lot_id] = new_price
        return new_price

print("✅ Model 1 (Baseline Linear) defined!")


✅ Model 1 (Baseline Linear) defined!


In [181]:
# ============================================================================
# MODEL 2 - DEMAND-BASED MODEL
# ============================================================================

class DemandBasedModel:
    def __init__(self, base_price=10.0):
        self.base_price = base_price
        self.current_prices = {}
        self.model_name = "Demand-Based Model"

        # Demand function parameters (can be tuned)
        self.params = {
            'alpha': 2.0,      # Occupancy weight
            'beta': 0.5,       # Queue length weight
            'gamma': 0.3,      # Traffic condition weight
            'delta': 0.8,      # Special day weight
            'epsilon': 0.2,    # Vehicle type weight
            'lambda': 0.5      # Price sensitivity
        }

        print(f"🎛️ {self.model_name} Parameters:")
        for param, value in self.params.items():
            print(f"   {param}: {value}")

    def initialize_prices(self, parking_lots):
        """Initialize prices for all parking lots"""
        for lot in parking_lots:
            self.current_prices[lot] = self.base_price
        print(f"✅ {self.model_name}: Initialized prices for {len(parking_lots)} lots")

    def calculate_demand(self, occupancy_rate, queue_length, traffic_condition,
                        is_special_day, vehicle_type_weight):
        """
        Calculate demand based on multiple factors
        """
        demand = (
            self.params['alpha'] * occupancy_rate +
            self.params['beta'] * queue_length -
            self.params['gamma'] * traffic_condition +
            self.params['delta'] * is_special_day +
            self.params['epsilon'] * vehicle_type_weight
        )
        return demand

    def normalize_demand(self, demand):
        """Normalize demand to prevent extreme pricing"""
        normalized = np.tanh(demand / 3.0)  # Scale factor of 3
        return normalized

    def predict_price(self, lot_id, occupancy_rate, queue_length, traffic_condition,
                     is_special_day, vehicle_type_weight):
        """
        Predict price based on demand function
        """
        # Calculate raw demand
        raw_demand = self.calculate_demand(
            occupancy_rate, queue_length, traffic_condition,
            is_special_day, vehicle_type_weight
        )

        # Normalize demand
        normalized_demand = self.normalize_demand(raw_demand)

        # Calculate price
        price_multiplier = 1 + self.params['lambda'] * normalized_demand
        new_price = self.base_price * price_multiplier

        # Apply bounds
        new_price = max(self.base_price * 0.5, min(new_price, self.base_price * 2.0))

        self.current_prices[lot_id] = new_price
        return new_price, raw_demand, normalized_demand

print("✅ Model 2 (Demand-Based) defined!")


✅ Model 2 (Demand-Based) defined!


In [182]:
# ============================================================================
# MODEL 3 - COMPETITIVE PRICING MODEL
# ============================================================================

class CompetitivePricingModel:
    def __init__(self, base_price=10.0):
        self.base_price = base_price
        self.current_prices = {}
        self.lot_locations = {}
        self.model_name = "Competitive Pricing Model"

        # Enhanced parameters
        self.params = {
            'alpha': 2.0,
            'beta': 0.5,
            'gamma': 0.3,
            'delta': 0.8,
            'epsilon': 0.2,
            'lambda': 0.5,
            'competition_weight': 0.3,  # Weight for competitor pricing
            'proximity_threshold': 0.01  # Distance threshold for competition
        }

        print(f"🎛️ {self.model_name} Parameters:")
        for param, value in self.params.items():
            print(f"   {param}: {value}")

    def initialize_prices(self, parking_lots):
        """Initialize prices for all parking lots"""
        for lot in parking_lots:
            self.current_prices[lot] = self.base_price
        print(f"✅ {self.model_name}: Initialized prices for {len(parking_lots)} lots")

    def update_lot_locations(self, lot_id, latitude, longitude):
        """Update location information for parking lots"""
        self.lot_locations[lot_id] = (latitude, longitude)

    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """Calculate approximate distance between two points"""
        return np.sqrt((lat2 - lat1)**2 + (lon2 - lon1)**2)

    def find_nearby_competitors(self, lot_id):
        """Find nearby parking lots (competitors)"""
        if lot_id not in self.lot_locations:
            return []

        current_lat, current_lon = self.lot_locations[lot_id]
        competitors = []

        for other_lot, (lat, lon) in self.lot_locations.items():
            if other_lot != lot_id:
                distance = self.calculate_distance(current_lat, current_lon, lat, lon)
                if distance <= self.params['proximity_threshold']:
                    competitors.append((other_lot, distance))

        return competitors

    def get_competitor_avg_price(self, competitors):
        """Calculate average price of nearby competitors"""
        if not competitors:
            return self.base_price

        total_price = 0
        total_weight = 0

        for lot_id, distance in competitors:
            price = self.current_prices.get(lot_id, self.base_price)
            weight = 1 / (distance + 0.001)  # Closer lots have higher weight
            total_price += price * weight
            total_weight += weight

        return total_price / total_weight if total_weight > 0 else self.base_price

    def predict_price(self, lot_id, occupancy_rate, queue_length, traffic_condition,
                     is_special_day, vehicle_type_weight, latitude, longitude):
        """
        Predict price considering competition
        """
        # Update location
        self.update_lot_locations(lot_id, latitude, longitude)

        # Calculate base demand
        raw_demand = (
            self.params['alpha'] * occupancy_rate +
            self.params['beta'] * queue_length -
            self.params['gamma'] * traffic_condition +
            self.params['delta'] * is_special_day +
            self.params['epsilon'] * vehicle_type_weight
        )

        # Find competitors and their prices
        competitors = self.find_nearby_competitors(lot_id)
        competitor_avg_price = self.get_competitor_avg_price(competitors)

        # Adjust demand based on competition
        if competitor_avg_price > self.base_price:
            competitive_adjustment = 0.2
        else:
            competitive_adjustment = -0.2

        adjusted_demand = raw_demand + competitive_adjustment
        normalized_demand = np.tanh(adjusted_demand / 3.0)

        # Calculate price
        base_price_from_demand = self.base_price * (1 + self.params['lambda'] * normalized_demand)

        # Blend with competitor pricing
        final_price = (
            (1 - self.params['competition_weight']) * base_price_from_demand +
            self.params['competition_weight'] * competitor_avg_price
        )

        # Apply bounds
        final_price = max(self.base_price * 0.5, min(final_price, self.base_price * 2.0))

        self.current_prices[lot_id] = final_price

        return final_price, raw_demand, normalized_demand, competitor_avg_price

print("✅ Model 3 (Competitive Pricing) defined!")


✅ Model 3 (Competitive Pricing) defined!


In [183]:
# ============================================================================
# REAL-TIME SIMULATION ENGINE
# ============================================================================

class RealTimeSimulator:
    def __init__(self, data_processor, model):
        self.data_processor = data_processor
        self.model = model
        self.results = []

    def simulate_real_time(self, data_subset=None, delay_seconds=0):
        """Simulate real-time pricing updates"""
        if data_subset is None:
            data_subset = self.data_processor.data.copy()

        # Sort by datetime for chronological simulation
        data_subset = data_subset.sort_values('DateTime')

        # Initialize model
        self.model.initialize_prices(self.data_processor.parking_lots)

        print(f"🚀 Starting real-time simulation with {self.model.model_name}")
        print(f"📊 Processing {len(data_subset)} records...")

        for idx, row in data_subset.iterrows():
            # Extract features
            lot_id = row['SystemCodeNumber']
            occupancy_rate = row['OccupancyRate']
            queue_length = row['QueueLength']
            traffic_condition = row['TrafficCondition_encoded']
            is_special_day = row['IsSpecialDay']
            vehicle_type_weight = row['VehicleType_encoded']

            # Get price prediction based on model type
            if isinstance(self.model, BaselineLinearModel):
                price = self.model.predict_price(lot_id, occupancy_rate)
                result = {
                    'DateTime': row['DateTime'],
                    'LotID': lot_id,
                    'Price': price,
                    'OccupancyRate': occupancy_rate,
                    'QueueLength': queue_length,
                    'TrafficCondition': row['TrafficConditionNearby'],
                    'VehicleType': row['VehicleType']
                }
            elif isinstance(self.model, DemandBasedModel):
                price, raw_demand, norm_demand = self.model.predict_price(
                    lot_id, occupancy_rate, queue_length, traffic_condition,
                    is_special_day, vehicle_type_weight
                )
                result = {
                    'DateTime': row['DateTime'],
                    'LotID': lot_id,
                    'Price': price,
                    'OccupancyRate': occupancy_rate,
                    'QueueLength': queue_length,
                    'TrafficCondition': row['TrafficConditionNearby'],
                    'VehicleType': row['VehicleType'],
                    'RawDemand': raw_demand,
                    'NormalizedDemand': norm_demand
                }
            elif isinstance(self.model, CompetitivePricingModel):
                price, raw_demand, norm_demand, comp_price = self.model.predict_price(
                    lot_id, occupancy_rate, queue_length, traffic_condition,
                    is_special_day, vehicle_type_weight, row['Latitude'], row['Longitude']
                )
                result = {
                    'DateTime': row['DateTime'],
                    'LotID': lot_id,
                    'Price': price,
                    'OccupancyRate': occupancy_rate,
                    'QueueLength': queue_length,
                    'TrafficCondition': row['TrafficConditionNearby'],
                    'VehicleType': row['VehicleType'],
                    'RawDemand': raw_demand,
                    'NormalizedDemand': norm_demand,
                    'CompetitorPrice': comp_price
                }

            self.results.append(result)

            # Show progress every 1000 records
            if len(self.results) % 1000 == 0:
                print(f"📈 Processed {len(self.results)} records...")

        print(f"✅ Simulation complete! Processed {len(self.results)} records.")
        return pd.DataFrame(self.results)

print("✅ Real-Time Simulation Engine defined!")


✅ Real-Time Simulation Engine defined!


In [184]:
# ============================================================================
# BOKEH VISUALIZATION MODULE
# ============================================================================

class BokehPricingVisualizer:
    def __init__(self):
        self.colors = Category10[10]
        self.tools = "pan,wheel_zoom,box_zoom,reset,save"

    def plot_price_trends(self, results_df, lot_ids=None, title_suffix=""):
        """Plot price trends over time using Bokeh"""
        if lot_ids is None:
            lot_ids = results_df['LotID'].unique()[:5]  # Show first 5 lots

        p = figure(
            title=f'Dynamic Pricing Trends Over Time {title_suffix}',
            x_axis_label='Time',
            y_axis_label='Price ($)',
            width=900,
            height=400,
            tools=self.tools,
            x_axis_type='datetime'
        )

        for i, lot_id in enumerate(lot_ids):
            lot_data = results_df[results_df['LotID'] == lot_id]
            if len(lot_data) > 0:
                source = ColumnDataSource(lot_data)
                p.line(
                    'DateTime', 'Price',
                    source=source,
                    legend_label=f'Lot {lot_id}',
                    line_color=self.colors[i % len(self.colors)],
                    line_width=2,
                    alpha=0.8
                )

                # Add hover tool
                hover = HoverTool(
                    tooltips=[
                        ('DateTime', '@DateTime{%F %T}'),
                        ('Lot ID', '@LotID'),
                        ('Price', '$@Price{0.00}'),
                        ('Occupancy Rate', '@OccupancyRate{0.00}'),
                        ('Vehicle Type', '@VehicleType')
                    ],
                    formatters={'@DateTime': 'datetime'},
                    mode='vline'
                )
                p.add_tools(hover)

        p.legend.location = "top_left"
        p.legend.click_policy = "hide"

        show(p)

    def plot_occupancy_vs_price(self, results_df, title_suffix=""):
        """Plot occupancy rate vs price relationship using Bokeh"""
        p = figure(
            title=f'Occupancy Rate vs Price {title_suffix}',
            x_axis_label='Occupancy Rate',
            y_axis_label='Price ($)',
            width=600,
            height=400,
            tools=self.tools
        )

        source = ColumnDataSource(results_df)
        p.scatter(
            'OccupancyRate', 'Price',
            source=source,
            size=6,
            alpha=0.6,
            color='blue'
        )

        # Add trend line
        slope, intercept = np.polyfit(results_df['OccupancyRate'], results_df['Price'], 1)
        x_trend = np.linspace(results_df['OccupancyRate'].min(), results_df['OccupancyRate'].max(), 100)
        y_trend = slope * x_trend + intercept
        p.line(x_trend, y_trend, line_color='red', line_width=2, alpha=0.8, legend_label='Trend Line')

        # Add hover tool
        hover = HoverTool(
            tooltips=[
                ('Occupancy Rate', '@OccupancyRate{0.00}'),
                ('Price', '$@Price{0.00}'),
                ('Lot ID', '@LotID'),
                ('Vehicle Type', '@VehicleType')
            ]
        )
        p.add_tools(hover)

        p.legend.location = "top_left"
        show(p)

    def plot_demand_analysis(self, results_df, title_suffix=""):
        """Plot demand analysis using Bokeh"""
        if 'RawDemand' in results_df.columns:
            p1 = figure(
                title=f'Raw Demand vs Price {title_suffix}',
                x_axis_label='Raw Demand',
                y_axis_label='Price ($)',
                width=450,
                height=400,
                tools=self.tools
            )

            source1 = ColumnDataSource(results_df)
            p1.scatter(
                'RawDemand', 'Price',
                source=source1,
                size=6,
                alpha=0.6,
                color='green'
            )

            hover1 = HoverTool(
                tooltips=[
                    ('Raw Demand', '@RawDemand{0.00}'),
                    ('Price', '$@Price{0.00}'),
                    ('Lot ID', '@LotID')
                ]
            )
            p1.add_tools(hover1)

            p2 = figure(
                title=f'Normalized Demand vs Price {title_suffix}',
                x_axis_label='Normalized Demand',
                y_axis_label='Price ($)',
                width=450,
                height=400,
                tools=self.tools
            )

            source2 = ColumnDataSource(results_df)
            p2.scatter(
                'NormalizedDemand', 'Price',
                source=source2,
                size=6,
                alpha=0.6,
                color='orange'
            )

            hover2 = HoverTool(
                tooltips=[
                    ('Normalized Demand', '@NormalizedDemand{0.00}'),
                    ('Price', '$@Price{0.00}'),
                    ('Lot ID', '@LotID')
                ]
            )
            p2.add_tools(hover2)

            grid = gridplot([[p1, p2]])
            show(grid)

    def plot_summary_statistics(self, results_df, model_name):
        """Plot summary statistics using Bokeh"""

        # 1. Price distribution histogram
        hist, edges = np.histogram(results_df['Price'], bins=30)
        p1 = figure(
            title='Price Distribution',
            x_axis_label='Price ($)',
            y_axis_label='Frequency',
            width=450,
            height=300,
            tools=self.tools
        )
        p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
                fill_color='skyblue', line_color='white', alpha=0.7)

        # 2. Price by vehicle type (box plot approximation)
        vehicle_types = results_df['VehicleType'].unique()
        vehicle_stats = []

        for vt in vehicle_types:
            vt_prices = results_df[results_df['VehicleType'] == vt]['Price']
            q1 = vt_prices.quantile(0.25)
            q2 = vt_prices.quantile(0.5)
            q3 = vt_prices.quantile(0.75)
            vehicle_stats.append({
                'vehicle_type': vt,
                'q1': q1,
                'q2': q2,
                'q3': q3,
                'mean': vt_prices.mean()
            })

        vehicle_df = pd.DataFrame(vehicle_stats)

        p2 = figure(
            title='Price by Vehicle Type',
            x_range=vehicle_types,
            x_axis_label='Vehicle Type',
            y_axis_label='Price ($)',
            width=450,
            height=300,
            tools=self.tools
        )

        source_vehicle = ColumnDataSource(vehicle_df)
        p2.scatter('vehicle_type', 'mean', source=source_vehicle, size=10, color='red', alpha=0.7)

        # 3. Price by traffic condition
        traffic_conditions = results_df['TrafficCondition'].unique()
        traffic_stats = []

        for tc in traffic_conditions:
            tc_prices = results_df[results_df['TrafficCondition'] == tc]['Price']
            traffic_stats.append({
                'traffic_condition': tc,
                'mean_price': tc_prices.mean(),
                'count': len(tc_prices)
            })

        traffic_df = pd.DataFrame(traffic_stats)

        p3 = figure(
            title='Price by Traffic Condition',
            x_range=traffic_conditions,
            x_axis_label='Traffic Condition',
            y_axis_label='Average Price ($)',
            width=450,
            height=300,
            tools=self.tools
        )

        source_traffic = ColumnDataSource(traffic_df)
        p3.vbar(x='traffic_condition', top='mean_price', source=source_traffic, width=0.5, color='lightblue')

        # 4. Queue length vs price scatter
        p4 = figure(
            title='Queue Length vs Price',
            x_axis_label='Queue Length',
            y_axis_label='Price ($)',
            width=450,
            height=300,
            tools=self.tools
        )

        source_queue = ColumnDataSource(results_df)
        p4.scatter('QueueLength', 'Price', source=source_queue, size=6, alpha=0.6, color='purple')

        hover4 = HoverTool(
            tooltips=[
                ('Queue Length', '@QueueLength'),
                ('Price', '$@Price{0.00}'),
                ('Lot ID', '@LotID')
            ]
        )
        p4.add_tools(hover4)

        # Create grid layout
        grid = gridplot([[p1, p2], [p3, p4]])
        show(grid)

    def plot_model_comparison(self, results_list, model_names):
        """Compare multiple models side by side"""
        p = figure(
            title='Model Comparison - Price Trends',
            x_axis_label='Record Index',
            y_axis_label='Price ($)',
            width=900,
            height=400,
            tools=self.tools
        )

        for i, (results_df, model_name) in enumerate(zip(results_list, model_names)):
            # Take average price across all lots for each record
            avg_prices = results_df.groupby(results_df.index)['Price'].mean()
            x_vals = list(range(len(avg_prices)))

            p.line(
                x_vals, avg_prices,
                legend_label=model_name,
                line_color=self.colors[i % len(self.colors)],
                line_width=2,
                alpha=0.8
            )

        p.legend.location = "top_left"
        p.legend.click_policy = "hide"

        show(p)

print("✅ Bokeh Visualization Module defined!")


✅ Bokeh Visualization Module defined!


In [185]:
# ============================================================================
# LOAD AND EXPLORE DATA
# ============================================================================

# Initialize data processor
print("🔧 Initializing Data Processor...")
processor = ParkingDataProcessor()

# Load data
print("📂 Loading dataset...")
if processor.load_data('dataset.csv'):
    print("✅ Data loaded successfully!")

    # Preprocess data
    print("\n🔄 Starting preprocessing...")
    if processor.preprocess_data():
        print("✅ Data preprocessing completed!")

        # Explore data
        print("\n📊 Exploring data...")
        processor.explore_data()

        print("\n🎯 Data is ready for modeling!")
        print("📋 Next: Run the individual model testing cells")
    else:
        print("❌ Preprocessing failed!")
else:
    print("❌ Failed to load data! Make sure 'dataset.csv' is in your Colab environment.")


🔧 Initializing Data Processor...
📂 Loading dataset...
✅ Data loaded successfully: (18368, 12)
📊 Columns: ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']
✅ Data loaded successfully!

🔄 Starting preprocessing...
🔄 Starting data preprocessing...
✅ DateTime column created
✅ Occupancy rate calculated
✅ Categorical variables encoded
✅ Time features extracted
✅ Found 14 unique parking lots
🎉 Preprocessing complete!
✅ Data preprocessing completed!

📊 Exploring data...

📊 DATA EXPLORATION SUMMARY
📈 Dataset shape: (18368, 18)
📅 Date range: 2016-10-04 07:59:00 to 2016-12-19 16:30:00
🏢 Number of parking lots: 14

🚗 Parking Lots: ['BHMBCCMKT01', 'BHMBCCTHL01', 'BHMEURBRD01', 'BHMMBMMBX01', 'BHMNCPHST01', 'BHMNCPNST01', 'Broad Street', 'Others-CCCPS105a', 'Others-CCCPS119a', 'Others-CCCPS135a', 'Others-CCCPS202', 'Others-CCCPS8', 'Others-CCCPS98', 'Shopping']

📊 

In [186]:
# ============================================================================
# TEST MODEL 1 - BASELINE LINEAR MODEL
# ============================================================================

print("\n" + "="*60)
print("🎯 TESTING MODEL 1: BASELINE LINEAR MODEL")
print("="*60)

# Initialize model
model1 = BaselineLinearModel(base_price=10.0, alpha=5.0)
print(f"📊 Base price: ${model1.base_price}")
print(f"⚙️ Alpha (sensitivity): {model1.alpha}")

# Initialize simulator
simulator1 = RealTimeSimulator(processor, model1)

# Run simulation on subset (first 500 records for quick testing)
print("\n🚀 Running simulation...")
results1 = simulator1.simulate_real_time(
    data_subset=processor.data.head(500),
    delay_seconds=0
)

# Show results
print("\n📈 Model 1 Results Summary:")
print(f"Records processed: {len(results1)}")
print(f"Price range: ${results1['Price'].min():.2f} - {results1['Price'].max():.2f}")
print(f"Average price: ${results1['Price'].mean():.2f}")

print("\n📊 Sample predictions:")
print(results1.head(10)[['DateTime', 'LotID', 'Price', 'OccupancyRate', 'VehicleType']].to_string())

# Initialize visualizer
visualizer = BokehPricingVisualizer()

print("\n📈 Generating visualizations...")
visualizer.plot_price_trends(results1, title_suffix="- Model 1 (Baseline)")
visualizer.plot_occupancy_vs_price(results1, title_suffix="- Model 1 (Baseline)")
visualizer.plot_summary_statistics(results1, "Model 1: Baseline Linear")

print("✅ Model 1 testing complete!")


🎯 TESTING MODEL 1: BASELINE LINEAR MODEL
📊 Base price: $10.0
⚙️ Alpha (sensitivity): 5.0

🚀 Running simulation...
✅ Baseline Linear Model: Initialized prices for 14 lots
🚀 Starting real-time simulation with Baseline Linear Model
📊 Processing 500 records...
✅ Simulation complete! Processed 500 records.

📈 Model 1 Results Summary:
Records processed: 500
Price range: $10.53 - 20.00
Average price: $19.91

📊 Sample predictions:
             DateTime        LotID      Price  OccupancyRate VehicleType
0 2016-10-04 07:59:00  BHMBCCMKT01  10.528596       0.105719         car
1 2016-10-04 08:25:00  BHMBCCMKT01  11.083189       0.110919         car
2 2016-10-04 08:59:00  BHMBCCMKT01  11.776430       0.138648         car
3 2016-10-04 09:32:00  BHMBCCMKT01  12.703640       0.185442         car
4 2016-10-04 09:59:00  BHMBCCMKT01  14.003466       0.259965        bike
5 2016-10-04 10:26:00  BHMBCCMKT01  15.537262       0.306759         car
6 2016-10-04 10:59:00  BHMBCCMKT01  17.435009       0.379549 

✅ Model 1 testing complete!


In [187]:
# ============================================================================
# TEST MODEL 2 - DEMAND-BASED MODEL (with Bokeh Visualization)
# ============================================================================

print("\n" + "="*60)
print("🎯 TESTING MODEL 2: DEMAND-BASED MODEL (with Bokeh)")
print("="*60)

# Initialize model
model2 = DemandBasedModel(base_price=10.0)

# Initialize simulator
simulator2 = RealTimeSimulator(processor, model2)

# Run simulation on subset (first 500 records for quick testing)
print("\n🚀 Running simulation...")
results2 = simulator2.simulate_real_time(
    data_subset=processor.data.head(500),
    delay_seconds=0
)

# Show results
print("\n📈 Model 2 Results Summary:")
print(f"Records processed: {len(results2)}")
print(f"Price range: ${results2['Price'].min():.2f} - ${results2['Price'].max():.2f}")
print(f"Average price: ${results2['Price'].mean():.2f}")
print(f"Demand range: {results2['RawDemand'].min():.2f} - {results2['RawDemand'].max():.2f}")

print("\n📊 Sample predictions:")
print(results2.head(10)[['DateTime', 'LotID', 'Price', 'OccupancyRate', 'RawDemand', 'NormalizedDemand']].to_string())

# Initialize Bokeh visualizer
bokeh_visualizer = BokehPricingVisualizer()

print("\n📈 Generating Bokeh visualizations...")
bokeh_visualizer.plot_price_trends(results2, title_suffix="- Model 2 (Demand-Based)")
bokeh_visualizer.plot_occupancy_vs_price(results2, title_suffix="- Model 2 (Demand-Based)")
bokeh_visualizer.plot_demand_analysis(results2, title_suffix="- Model 2 (Demand-Based)")
bokeh_visualizer.plot_summary_statistics(results2, "Model 2: Demand-Based")

print("✅ Model 2 testing complete!")


🎯 TESTING MODEL 2: DEMAND-BASED MODEL (with Bokeh)
🎛️ Demand-Based Model Parameters:
   alpha: 2.0
   beta: 0.5
   gamma: 0.3
   delta: 0.8
   epsilon: 0.2
   lambda: 0.5

🚀 Running simulation...
✅ Demand-Based Model: Initialized prices for 14 lots
🚀 Starting real-time simulation with Demand-Based Model
📊 Processing 500 records...
✅ Simulation complete! Processed 500 records.

📈 Model 2 Results Summary:
Records processed: 500
Price range: $10.82 - $14.79
Average price: $13.08
Demand range: 0.49 - 5.76

📊 Sample predictions:
             DateTime        LotID      Price  OccupancyRate  RawDemand  NormalizedDemand
0 2016-10-04 07:59:00  BHMBCCMKT01  11.242497       0.105719   0.761438          0.248499
1 2016-10-04 08:25:00  BHMBCCMKT01  11.258744       0.110919   0.771837          0.251749
2 2016-10-04 08:59:00  BHMBCCMKT01  12.078292       0.138648   1.327296          0.415658
3 2016-10-04 09:32:00  BHMBCCMKT01  12.205630       0.185442   1.420884          0.441126
4 2016-10-04 09:59:

✅ Model 2 testing complete!


In [188]:
# ============================================================================
# TEST MODEL 3 - COMPETITIVE PRICING MODEL (with Bokeh Visualization)
# ============================================================================

print("\n" + "="*60)
print("🎯 TESTING MODEL 3: COMPETITIVE PRICING MODEL (with Bokeh)")
print("="*60)

# Initialize model
model3 = CompetitivePricingModel(base_price=10.0)

# Initialize simulator
simulator3 = RealTimeSimulator(processor, model3)

# Run simulation on subset (first 500 records for quick testing)
print("\n🚀 Running simulation...")
results3 = simulator3.simulate_real_time(
    data_subset=processor.data.head(500),
    delay_seconds=0
)

# Show results
print("\n📈 Model 3 Results Summary:")
print(f"Records processed: {len(results3)}")
print(f"Price range: ${results3['Price'].min():.2f} - ${results3['Price'].max():.2f}")
print(f"Average price: ${results3['Price'].mean():.2f}")
print(f"Competitor price range: ${results3['CompetitorPrice'].min():.2f} - ${results3['CompetitorPrice'].max():.2f}")

print("\n📊 Sample predictions:")
print(results3.head(10)[['DateTime', 'LotID', 'Price', 'OccupancyRate', 'CompetitorPrice']].to_string())

# Initialize Bokeh visualizer
bokeh_visualizer = BokehPricingVisualizer()

print("\n📈 Generating Bokeh visualizations...")
bokeh_visualizer.plot_price_trends(results3, title_suffix="- Model 3 (Competitive)")
bokeh_visualizer.plot_occupancy_vs_price(results3, title_suffix="- Model 3 (Competitive)")
bokeh_visualizer.plot_demand_analysis(results3, title_suffix="- Model 3 (Competitive)")
bokeh_visualizer.plot_summary_statistics(results3, "Model 3: Competitive Pricing")

print("✅ Model 3 testing complete!")


🎯 TESTING MODEL 3: COMPETITIVE PRICING MODEL (with Bokeh)
🎛️ Competitive Pricing Model Parameters:
   alpha: 2.0
   beta: 0.5
   gamma: 0.3
   delta: 0.8
   epsilon: 0.2
   lambda: 0.5
   competition_weight: 0.3
   proximity_threshold: 0.01

🚀 Running simulation...
✅ Competitive Pricing Model: Initialized prices for 14 lots
🚀 Starting real-time simulation with Competitive Pricing Model
📊 Processing 500 records...
✅ Simulation complete! Processed 500 records.

📈 Model 3 Results Summary:
Records processed: 500
Price range: $10.34 - $13.33
Average price: $12.02
Competitor price range: $10.00 - $10.00

📊 Sample predictions:
             DateTime        LotID      Price  OccupancyRate  CompetitorPrice
0 2016-10-04 07:59:00  BHMBCCMKT01  10.647470       0.105719             10.0
1 2016-10-04 08:25:00  BHMBCCMKT01  10.659179       0.110919             10.0
2 2016-10-04 08:59:00  BHMBCCMKT01  11.256585       0.138648             10.0
3 2016-10-04 09:32:00  BHMBCCMKT01  11.350613       0.18544

✅ Model 3 testing complete!


In [189]:
# ============================================================================
# COMPARE ALL MODELS (with Bokeh Visualization)
# ============================================================================

print("\n" + "="*60)
print("🔍 COMPARING ALL THREE MODELS (with Bokeh)")
print("="*60)

# Create comparison summary
comparison_data = {
    'Model': ['Baseline Linear', 'Demand-Based', 'Competitive'],
    'Average Price ($)': [results1['Price'].mean(), results2['Price'].mean(), results3['Price'].mean()],
    'Price Range ($)': [f"{results1['Price'].min():.2f} - {results1['Price'].max():.2f}",
                      f"{results2['Price'].min():.2f} - {results2['Price'].max():.2f}",
                      f"{results3['Price'].min():.2f} - {results3['Price'].max():.2f}"],
    'Records Processed': [len(results1), len(results2), len(results3)]
}

comparison_df = pd.DataFrame(comparison_data)

print("\n📊 Model Comparison Summary:")
display(comparison_df)

print("\n📈 Generating Bokeh comparison visualization...")
# Initialize Bokeh visualizer
bokeh_visualizer = BokehPricingVisualizer()
bokeh_visualizer.plot_model_comparison([results1, results2, results3], ['Baseline Linear', 'Demand-Based', 'Competitive'])


print("\n✅ Model comparison complete!")
print("🎉 Analysis complete! Explore the results and visualizations above.")


🔍 COMPARING ALL THREE MODELS (with Bokeh)

📊 Model Comparison Summary:


,Model,Average Price ($),Price Range ($),Records Processed
0,Baseline Linear,19.905286,10.53 - 20.00,500
1,Demand-Based,13.077649,10.82 - 14.79,500
2,Competitive,12.015005,10.34 - 13.33,500



📈 Generating Bokeh comparison visualization...



✅ Model comparison complete!
🎉 Analysis complete! Explore the results and visualizations above.


In [190]:
# ============================================================================
# REALTIME STREAMING PLOT (with Bokeh Visualization)
# ============================================================================
selected_results_df = results3
source = ColumnDataSource(selected_results_df)
p = figure(
    title="Real-Time Price Simulation (Competitive Pricing Model)",
    x_axis_label='Time',
    y_axis_label='Price ($)',
    width=900,
    height=400,
    tools="pan,wheel_zoom,box_zoom,reset,save", # Using the default tools string
    x_axis_type='datetime'
)

hover = HoverTool(
    tooltips=[
        ('DateTime', '@DateTime{%F %T}'),
        ('Lot ID', '@LotID'),
        ('Price', '$@Price{0.00}'),
        ('Occupancy Rate', '@OccupancyRate{0.00}'),
        ('Vehicle Type', '@VehicleType')
    ],
    formatters={'@DateTime': 'datetime'},
    mode='vline'
)
p.add_tools(hover)
p.line(
    'DateTime', 'Price',
    source=source,
    legend_label='Price Trend',
    line_color='blue',
    line_width=2,
    alpha=0.8
)

p.scatter(
    'DateTime', 'Price',
    source=source,
    size=6,
    alpha=0.6,
    color='red'
)

GlyphRenderer(id='d30fdae5-7dc4-47fc-9c10-6591b054e7d4', ...)

In [191]:
import time
from bokeh.io import push_notebook

# Assuming 'source' and 'p' from previous cells are available
# Assuming 'selected_results_df' from the previous step is available

# Clear the existing data in the source to start the simulation from the beginning
source.data = {col: [] for col in selected_results_df.columns}

# Show the initial empty plot
handle = show(p, notebook_handle=True)

# Simulate real-time updates
for i in range(len(selected_results_df)):
    # Get the next row of data
    new_data = selected_results_df.iloc[[i]]

    # Append the new data to the ColumnDataSource
    # Ensure the data is in a dictionary format with lists as values
    source.stream({col: new_data[col].tolist() for col in selected_results_df.columns})

    # Push the updates to the plot
    push_notebook(handle=handle)

    # Add a small delay to simulate real-time streaming
    time.sleep(0.1) # Adjust the delay as needed

print("✅ Real-time simulation complete!")

✅ Real-time simulation complete!


# Dynamic Parking Pricing Models Report

## Introduction/Project Overview

This report details the development and evaluation of different dynamic pricing models for urban parking lots. The core **goal** is to implement and compare models that adjust parking prices in real-time based on various factors. By dynamically adjusting prices based on demand and other relevant conditions, the aim is to optimize parking lot **occupancy** and **revenue**, providing a more efficient parking system for both operators and users. The project employs a **data-driven approach**, simulating real-time scenarios to test the effectiveness of each pricing strategy.

## Data Processing Steps

A custom `ParkingDataProcessor` class (defined in **Cell 2**) was used to load, clean, and prepare the raw parking data (`dataset.csv`). This class encapsulates the essential steps required to transform the raw data into a format suitable for the pricing models. Key preprocessing steps included:

1.  **Creating a `DateTime` column:** Combining the `LastUpdatedDate` and `LastUpdatedTime` columns into a single, properly formatted datetime object. This is crucial for analyzing time-series data and simulating real-time updates chronologically.
2.  **Calculating `OccupancyRate`:** Computing the ratio of `Occupancy` to `Capacity` for each record. This provides a standardized metric (ranging from 0 to 1 or slightly above if over capacity) that is a primary indicator of parking demand.
3.  **Encoding Categorical Variables:** Converting the `VehicleType` and `TrafficConditionNearby` columns into numerical representations (`VehicleType_encoded`, `TrafficCondition_encoded`). This allows these qualitative features to be used in mathematical models, assigning weights based on their assumed relative impact on demand (e.g., trucks might take more space or time, high traffic might deter drivers).
4.  **Extracting Time Features:** Deriving `Hour` and `DayOfWeek` from the `DateTime` column. These features capture temporal patterns in parking demand, as needs and traffic conditions often vary significantly by hour of the day and day of the week.

These steps ensure the data is clean, consistent, and contains the necessary features for the pricing models to function effectively.

## Model Descriptions

Three different dynamic pricing models were implemented and tested:

### Baseline Linear Model

This is the simplest model (implemented in **Cell 3**). It's a basic **reactive pricing** strategy. The price is adjusted solely based on the `OccupancyRate`. The core logic is a linear relationship: `Price(t+1) = Price(t) + α * OccupancyRate`. The `alpha` parameter (sensitivity) determines how aggressively the price changes in response to occupancy. A higher `alpha` means prices will increase or decrease more sharply with changes in occupancy. The model also includes **price bounds** to prevent extreme values, ensuring the price stays within a defined range (0.5x to 2x the `base_price`).

### Demand-Based Model

This model (implemented in **Cell 4**) adopts a more sophisticated **demand-responsive pricing** approach. Instead of relying only on occupancy, it calculates a composite **'raw demand' score** based on a weighted linear combination of multiple factors:
-   `OccupancyRate` (weight `alpha`)
-   `QueueLength` (weight `beta`)
-   `TrafficCondition_encoded` (weight `gamma`)
-   `IsSpecialDay` (weight `delta`)
-   `VehicleType_encoded` (weight `epsilon`)

The weights (`alpha`, `beta`, `gamma`, `delta`, `epsilon`) are parameters that can be tuned to reflect the perceived importance of each factor in driving demand. A negative weight is applied to `TrafficCondition_encoded` because high traffic is assumed to *decrease* the desirability (and thus potentially the effective demand) for parking nearby due to difficulty in access.

The raw demand score is then **normalized** using the hyperbolic tangent (`tanh`) function. This squashes the raw demand value into a range between -1 and 1, preventing the price calculation from becoming overly sensitive to extreme raw demand values. The final price is calculated as `Base Price * (1 + lambda * Normalized Demand)`, where `lambda` is a **price sensitivity** parameter controlling how much the price reacts to the normalized demand. This model also enforces the same **price bounds** as the baseline model.

### Competitive Pricing Model

This model (implemented in **Cell 5**) extends the **demand-based approach** by incorporating the concept of **competitive pricing**. It recognizes that parking lots don't operate in isolation and that nearby competitors' prices can influence customer choice and thus, the optimal price.

It first calculates a demand-based price similar to the Demand-Based Model. However, it then **finds nearby competitors** by checking the `Latitude` and `Longitude` of other parking lots within a specified `proximity_threshold` (using a simplified Euclidean distance). It calculates the **average price of these nearby competitors**, potentially using a weighted average where closer competitors have more influence.

The final price is a **blend** of the demand-based price and the competitor average price. The `competition_weight` parameter determines the balance; a higher weight means the model is more influenced by competitor pricing, while a lower weight means it relies more on its internal demand calculation. The logic involves a slight adjustment to the raw demand based on whether the competitor average price is higher or lower than the base price, before blending the resulting demand-based price with the competitor average price. This model also includes the same **price bounds**.

## Demand Function Explanation

In the **Demand-Based** and **Competitive Pricing Models**, the core of the pricing logic is the calculation of a **raw demand score**. This is achieved through a **weighted linear combination** of several input features:

`Raw Demand = (alpha * Occupancy Rate) + (beta * Queue Length) - (gamma * Traffic Condition) + (delta * Is Special Day) + (epsilon * Vehicle Type Weight)`

-   **`alpha` (Occupancy Rate):** A positive weight. Higher occupancy suggests higher current demand or desirability, increasing the raw demand score.
-   **`beta` (Queue Length):** A positive weight. A longer queue indicates strong immediate demand exceeding current capacity, significantly increasing the raw demand score.
-   **`gamma` (Traffic Condition):** A positive weight, but applied with a *negative* sign in the sum. Indicates encoded traffic condition (e.g., low, average, high). High traffic nearby makes the lot harder to access, potentially reducing effective demand. The negative sign ensures higher traffic scores *decrease* the raw demand.
-   **`delta` (Is Special Day):** A positive weight. Special days (like holidays or events) are assumed to increase overall demand for parking, adding to the raw demand score.
-   **`epsilon` (Vehicle Type Weight):** A positive weight. Different vehicle types might have different impacts on lot turnover or space utilization, influencing the demand calculation. The encoded weights reflect these assumed differences.

The resulting `Raw Demand` score is a single number intended to capture the aggregate pressure on the parking lot at a given moment.

This raw score is then passed through a **normalization step** using the `tanh(x / scale_factor)` function. Normalization is essential because the raw demand can vary significantly depending on the input values and weights. `tanh` squashes any input value into the range (-1, 1). This prevents extremely high or low raw demand scores from resulting in unboundedly high or low price multipliers, ensuring the final price remains within a predictable and bounded range, making the model more stable and preventing unrealistic price predictions.

## Assumptions Made

Several assumptions underpin the design and simulation of these models:

-   **Data Accuracy:** It is assumed that the provided `dataset.csv` accurately captures real-world parking conditions, including occupancy, queue length, traffic, and special day information, without significant errors or biases.
-   **Encoded Value Representation:** The numerical values assigned during categorical encoding (`VehicleType_encoded`: car=1.0, bike=0.5, truck=1.5, cycle=0.3; `TrafficCondition_encoded`: low=0.5, average=1.0, high=1.5) are assumed to accurately reflect the *relative impact* of these categories on parking demand. For example, a truck is assumed to contribute more to "demand pressure" (perhaps due to space or maneuverability) than a car, and high traffic is assumed to have a stronger negative impact than low traffic.
-   **Parameter Appropriateness:** The specific numerical values chosen for the model parameters (`alpha`, `beta`, `gamma`, `delta`, `epsilon`, `lambda`, `competition_weight`, `proximity_threshold`) are assumed to be reasonable starting points or approximations of how these factors *should* influence pricing in a real urban environment. In a production system, these parameters would ideally be learned from historical data or expert knowledge.
-   **Linear Relationships:** The demand function assumes a primarily **linear relationship** between the weighted sum of features and the raw demand. While normalized later, the initial aggregation is linear. This is a simplification; real-world demand might have more complex, non-linear relationships with these factors.
-   **Simplified Competition:** The Competitive Pricing Model uses a simplified distance calculation and assumes that competitor influence can be captured by a weighted average of nearby prices within a fixed radius and a simple binary adjustment to raw demand based on whether competitor prices are above the base price. Real competition might involve more complex dynamics and market segmentation.
-   **Real-time Simulation Fidelity:** The simulation engine processes records sequentially by timestamp, mimicking a real-time data stream. It assumes that the model's decision at one timestamp immediately influences the 'state' (current price) for the next relevant record, regardless of the actual time gap.

## Price Changes with Demand and Competition

The way price changes in each model directly reflects its underlying logic:

-   **Baseline Linear Model:** Price changes are **directly and linearly proportional** to the **Occupancy Rate**. If occupancy increases, price increases; if occupancy decreases, price decreases. The magnitude of this change is scaled by the `alpha` parameter. It does *not* consider queue length, traffic, special days, vehicle types, or competitor prices.
-   **Demand-Based Model:** Price changes are driven by the **composite Normalized Demand score**. As the calculated raw demand (influenced by all five factors: occupancy, queue, traffic, special day, vehicle type) increases, the normalized demand increases (towards 1), leading to a higher price. Conversely, as raw demand decreases, normalized demand decreases (towards -1), leading to a lower price. The `lambda` parameter controls the sensitivity of price to this normalized demand. This model is more responsive to a broader set of conditions than the baseline.
-   **Competitive Pricing Model:** Price changes are influenced by **both the calculated demand factors AND the prices of nearby competitors**. The model first calculates a demand-based price. It then considers the average competitor price. The final price is a weighted average of these two values.
    -   If the demand factors suggest a high price, but competitors are priced significantly lower, the `competition_weight` will pull the final price downwards to remain competitive.
    -   If demand is low, but competitors are priced higher, the `competition_weight` might pull the price upwards slightly compared to a pure demand-based approach, potentially allowing the lot to capture some demand from the higher-priced competitors.
    -   The model dynamically balances responding to local conditions (demand) with reacting to the external market (competitors).

## Model Testing and Comparison

Each of the three models (`BaselineLinearModel`, `DemandBasedModel`, `CompetitivePricingModel`) was tested using the `RealTimeSimulator` (defined in **Cell 6**). To facilitate quicker testing and visualization, the simulation was run on a **subset** of the processed data (the first 500 records in the executed cells). This simulation mimics a real-time scenario by processing data records chronologically.

The simulation results for each model are stored in separate DataFrames (`results1`, `results2`, `results3`). These DataFrames contain the predicted price at each timestamp, along with the input features and, for the more complex models, intermediate calculations like raw/normalized demand and competitor price.

The **comparison** focuses on analyzing the characteristics of the prices generated by each model, including:
-   Overall **Average Price** and **Price Range** observed during the simulation (`comparison_df`).
-   How prices **fluctuate over time** (visualized price trends).
-   The **relationship** between key input features (like occupancy or demand) and the predicted price.
-   The **distribution** of prices generated by each model.

By comparing these aspects across the three models using the simulation results and the `comparison_df`, we can evaluate their respective behaviors and potential strengths and weaknesses in a dynamic environment.

## Visualizations

The `BokehPricingVisualizer` class (defined in **Cell 7**) is used to generate interactive plots that help understand the models' behavior:

-   **Price Trends Over Time:** Plots showing how the predicted price for selected parking lots (or the average price across all lots for comparison) changes over the sequence of records processed. This helps visualize price volatility, responsiveness, and overall price levels for each model.
-   **Occupancy Rate vs Price:** A scatter plot showing the relationship between the occupancy rate and the predicted price. This highlights how directly and strongly each model links pricing to this key metric. The trend line provides a summary of this relationship.
-   **Demand Analysis (Raw/Normalized Demand vs Price):** Scatter plots (for Demand-Based and Competitive models) showing how the calculated raw and normalized demand scores relate to the predicted price. This illustrates the effectiveness of the demand function and normalization in driving price adjustments.
-   **Summary Statistics Plots:** A grid of plots providing an overview:
    -   A histogram of the price distribution shows the range and frequency of different price points generated by the model.
    -   Plots of average price by vehicle type and traffic condition show how these categorical factors implicitly (via the demand function) or explicitly influence pricing decisions on average.
    -   A scatter plot of Queue Length vs Price shows the model's responsiveness to queue size.
-   **Model Comparison - Price Trends:** A line plot comparing the average price across all lots for each model over the simulation records. This provides a high-level view of how the overall price level evolves differently under each strategy.

These visualizations are crucial for qualitatively assessing the models' behavior and comparing their dynamic responses to changing parking conditions.

## Conclusion

Based on the simulation results, the price trends observed, the relationship between price and influencing factors, and the summary statistics presented in the visualizations, this section will summarize the key findings for each model. It will discuss which model appears most promising for dynamic parking pricing based on the simulated data and the defined objectives, and outline potential future work, such as parameter tuning, incorporating more data features, or exploring more complex modeling techniques.